# Taller: Predicción de series temporales usando Deep Learning

### Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa import stattools
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

### Carga de datos

In [ ]:
data = pd.read_csv('household_power_consumption.txt', sep=';', header=0,
                  low_memory=False, infer_datetime_format=True,
                  parse_dates={'datetime':[0,1]}, index_col=['datetime'])

In [ ]:
data.info()

In [ ]:
data.head()

### Preprocesamiento de datos

In [ ]:
data.isnull().sum()

In [ ]:
data.replace('?', np.nan, inplace=True)

In [ ]:
data.fillna(method='ffill', inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.astype('float32')

Para simplificar este problema, podemos generar un remuestreo de los datos a otro nivel de desagregación (actualmente los registros están por minuto).

En este caso, trabajaremos con las series por hora. Por lo tanto, cada serie debe ser transformada para que considere el total de energía por hora.

In [ ]:
dataH = data.resample('h').sum()

In [ ]:
data.shape, dataH.shape

Una primera aproximación será generada con la serie 'Global_active_power', para la cual se usarán como inputs los rezagos de la misma variable, esto es, modelaremos la serie de forma univariada.

In [ ]:
serie = dataH[['Global_active_power']]

In [ ]:
serie.shape

### Particionando los datos

In [ ]:
trainN = int(len(serie)*.2)

### Modelo Base

In [ ]:
plt.plot(stattools.acf(serie, nlags=24), 'bo')
plt.title('Función de Autocorrelación')
plt.xlabel('Lags')
plt.ylabel('ACF')
plt.show()

In [ ]:
plt.plot(stattools.pacf(serie, nlags=24), 'bo')
plt.title('Función de Autocorrelación Parcial')
plt.xlabel('Lags')
plt.ylabel('PACF')
plt.show()

En base al análisis gráfico de las funciones de autocorrelación y autocorrelación parcial, se infiere que la serie puede modelarse en base a un $sarima(1,0,1)\times(1,0,0)_{24}$

In [ ]:
sarima = SARIMAX(serie[:(-trainN+24)], order=(12,0,0), seasonal_order=(1,0,0,24))

In [ ]:
%%time
sarimaFit = sarima.fit(disp=-1)

In [ ]:
sarimaForecasth = sarimaFit.predict(len(serie)-trainN, len(serie)-trainN+24-1, dynamic=False).values

In [ ]:
sarimaForecastd = sarimaFit.predict(len(serie)-trainN, len(serie)-trainN+24-1, dynamic=True).values

In [ ]:
plt.plot(serie.values[-trainN:(-trainN+24)], label='Val Set')
plt.plot(sarimaForecasth, label='Sarima 1 hora')
plt.plot(sarimaForecastd, label='Sarima 1 día')
plt.legend()
plt.title('Predicción SARIMA')
plt.show()

In [ ]:
'Primeras 24 horas: %.2f' % np.sqrt(mean_squared_error(serie.values[-trainN:(-trainN+24)],sarimaForecasth))

In [ ]:
'Primeras 24 horas: %.2f' % np.sqrt(mean_squared_error(serie.values[-trainN:(-trainN+24)],sarimaForecastd))

### Modelo Supervisado

In [ ]:
def transformSerie(data, lag=1, fw=1):
    df = pd.DataFrame(data)
    #lags
    cols = [df.shift(i) for i in range(lag,0,-1)]
    #forwards
    cols += [df.shift(-i) for i in range(fw)]
    df = pd.concat(cols, axis=1)
    df.dropna(inplace=True)
    X, Y = df.iloc[:,:lag], df.iloc[:,lag:]
    return X,Y

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# Caso univariado con 24 lags
X, Y = transformSerie(serie, 24, 1)

In [ ]:
X.shape, Y.shape

Luego de transformada la serie, es necesario particionar los datos en un set de entrenamiento y otro de testeo. 

In [ ]:
trainX, trainY = X[:-trainN], Y[:-trainN]
testX, testY = X[-trainN:], Y[-trainN:]

### Redes Neuronales

In [ ]:
model = Sequential()

In [ ]:
model.add(layers.Dense(8, input_dim=trainX.shape[1]))
#model.add(layers.Dropout(.4))
#model.add(layers.Dense(4))
#model.add(layers.Dropout(.4))
model.add(layers.Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

Para volver a reproducir el mismo entrenmaiento, es necesario fijar una semilla.

In [ ]:
np.random.seed(1234)

In [ ]:
%%time
history = model.fit(trainX, trainY, batch_size=10, epochs=20, shuffle=False, validation_data=(testX, testY), verbose=0)
#history = model.fit(trainX, trainY, batch_size=10, epochs=20, shuffle=False, validation_data=(valX, valY), verbose=0, callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

In [ ]:
plt.plot(history.history['val_loss'], label='Validation loss', )
#plt.plot(history.history['loss'], label='Training loss')
plt.legend()
plt.show()

Ahora podemos revisar las predicciones que genera el modelo entrenado.

In [ ]:
NNforecasth = model.predict(testX, batch_size=10)

In [ ]:
plt.plot(testY[:(24)].values, label='Test set')
plt.plot(NNforecasth[:(24)], label='Forecast')
plt.legend()
plt.show()

Como medida de precisión del modelo a la hora de predecir, se calcula el rmse para el conjunto de muestreo y para el subconjunto de la primera semana.

In [ ]:
'Primeras 24 horas: %.2f' % np.sqrt(mean_squared_error(testY[:24], NNforecasth[:24]))

### Redes Neuronales Recurrentes

Para utilizar una red neuronal recurrente, por ejemplo LSTM, es necesario escalar el input y output, dado que la red ocupa como función de activación la tanh.

In [ ]:
scaler = MinMaxScaler((-1,1))

In [ ]:
scaler = scaler.fit(serie[:-trainN])

In [ ]:
serie_scaled = scaler.transform(serie)

In [ ]:
X, Y = transformSerie(serie_scaled, 1, 1)

In [ ]:
X.shape, Y.shape

In [ ]:
X = X.values.reshape(X.shape[0],1,X.shape[1])

In [ ]:
trainX, trainY = X[:-trainN], Y[:-trainN]
testX, testY = X[-trainN:], Y[-trainN:]

In [ ]:
model2 = Sequential()

In [ ]:
model2.add(layers.LSTM(1, input_shape=(trainX.shape[1], trainX.shape[2])))
model2.add(layers.Dense(1))

In [ ]:
model2.compile(optimizer='adam', loss='mse')

In [ ]:
model2.summary()

In [ ]:
%%time
history2 = model2.fit(trainX, trainY, batch_size=10, epochs=20, shuffle=False, verbose=0,
          validation_data=(testX, testY), callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

In [ ]:
plt.plot(history2.history['val_loss'], label='Validation loss', )
#plt.plot(history2.history['loss'], label='Training loss')
plt.legend()
plt.show()

In [ ]:
LSTMforecasth = scaler.inverse_transform(model2.predict(testX, batch_size=100))
testY_is = scaler.inverse_transform(testY)

In [ ]:
plt.plot(testY_is[:(24)], label='Test set')
plt.plot(LSTMforecasth[:(24)], label='Forecast LSTM')
plt.legend()
plt.show()

In [ ]:
'Primeras 24 horas: %.2f' % np.sqrt(mean_squared_error(testY_is[:24], LSTMforecasth[:24]))

## Proyección multi-step

In [ ]:
X, Y = transformSerie(serie, 24, 24)

In [ ]:
X.shape, Y.shape

In [ ]:
trainX, trainY = X[:(-trainN+23)], Y[:(-trainN+23)]
testX, testY = X[(-trainN+23):], Y[(-trainN+23):]

In [ ]:
model3 = Sequential()

In [ ]:
model3.add(layers.Dense(8, input_dim=(trainX.shape[1])))
#model3.add(layers.Dense(16))
model3.add(layers.Dense(24))

In [ ]:
model3.compile(optimizer='adam', loss='mse')

In [ ]:
model3.summary()

In [ ]:
%%time
history = model3.fit(trainX, trainY, batch_size=10, epochs=20, verbose=0, validation_data=(testX, testY),callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

In [ ]:
#plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], label='Validation loss', )
#plt.plot(history.history['loss'], label='Training loss')
plt.legend()
plt.show()

In [ ]:
NNforecastd = model3.predict(testX, batch_size=10)

In [ ]:
NNforecastd = NNforecastd[0].reshape((NNforecastd[0].shape[0],1))

In [ ]:
plt.plot(testY[:1].values.transpose(), label='Test set')
plt.plot(NNforecastd, label='Forecast')
plt.legend()
plt.show()

In [ ]:
'Primeras 24 horas: %.2f' % np.sqrt(mean_squared_error(testY[:1].values.transpose(), NNforecastd))

In [ ]:
X, Y = transformSerie(serie_scaled, 1, 24)

In [ ]:
X.shape, Y.shape

In [ ]:
X = X.values.reshape(X.shape[0],1,X.shape[1])

In [ ]:
trainX, trainY = X[:(-trainN+23)], Y[:(-trainN+23)]
testX, testY = X[(-trainN+23):], Y[(-trainN+23):]

In [ ]:
model4 = Sequential()

In [ ]:
model4.add(layers.LSTM(4, input_shape=(trainX.shape[1], trainX.shape[2])))
#model4.add(layers.LSTM(1))
model4.add(layers.Dense(24))

In [ ]:
model4.compile(optimizer='adam', loss='mse')

In [ ]:
model4.summary()

In [ ]:
%%time
history4 = model4.fit(trainX, trainY, batch_size=10, epochs=20, shuffle=False, verbose=0,
          validation_data=(testX, testY), callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

In [ ]:
plt.plot(history4.history['val_loss'], label='Validation loss', )
#plt.plot(history2.history['loss'], label='Training loss')
plt.legend()
plt.show()

In [ ]:
LSTMforecastd = scaler.inverse_transform(model4.predict(testX, batch_size=10))
testY_is = scaler.inverse_transform(testY)

In [ ]:
plt.plot(testY_is[:1].transpose(), label='Test set')
plt.plot(LSTMforecastd[:1].transpose(), label='Forecast LSTM')
#plt.plot(Yhat3[:1].transpose(), label='Forecast NN')
plt.legend()
plt.show()

In [ ]:
'Primeras 24 horas: %.2f' % np.sqrt(mean_squared_error(testY_is[:1].transpose(), LSTMforecastd[:1].transpose()))